### Imports

In [1]:
import requests
import json
import ee
import geemap
import folium
import os
import pandas as pd
import glob
print("imports complete")

imports complete


### Web app deployed URLs

In [2]:
asset_registry_base = "https://api-ar.agstack.org"
user_registry_base = "https://user-registry.agstack.org"

### Add your email & password to register with Agstack and test out the APIs

In [3]:
email = "my_name@domain.org"
phone_num = "0123456789"
password = "this_is_not_a_password"
discoverable = True
##appears to not need proper credentials for running this...

#### Start Session

In [4]:
# using session to store cookies that are persistent
session = requests.session()
session.headers = headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json'
}
req_body = {'email': email, 'password': password}
res = session.post(user_registry_base, json=req_body)
print(session.cookies)
print(res.status_code)

<RequestsCookieJar[]>
500


In [5]:
ee.Initialize()

### Functions

In [6]:
#Function to take nominal scale from template image and set as "scale" property in target image. 
#useful for composite images (e.g., mosaics) where original scale is lost
def set_scale_property_from_image(target_image,template_image,template_band_index=0,verbose=False):
    out_scale = get_scale_from_image(template_image,band_index=template_band_index)
    if verbose:
        print("template_band_index: ",template_band_index)
        print("scale (m): ",out_scale)
    output_image = target_image.set("scale",out_scale)
    return output_image

#gets nominal scale from image (NB this should not be from a composite/mosaic or incorrrect value returned)
def get_scale_from_image(image,band_index=0):
    return image.select(band_index).projection().nominalScale().getInfo()

# get pixel area in km2 for image
def binary_to_area_km2(image):
    return image.multiply(ee.Image.pixelArea()).divide(1e6) 

# add area property in km2 for input feature (ROI)
def add_area_km2_property_to_feature (feature):
    feature = feature.set("area_km2",feature.area().divide(1e6))#add area
    # if verbose:
    #     print(feature.get("area_km2").getInfo())
    return feature


# ##API geo_id to ee feature
# def geo_id_to_feature(geo_id):
#     res = session.get(asset_registry_base + f"/fetch-field/{geo_id}?s2_index=") # s2 index are indexes for which we need S2 cell token
#     poly_json = res.json()['Geo JSON']['geometry']['coordinates']
#     feature = ee.Feature(ee.Geometry.Polygon(poly_json),ee.Dictionary(["geoid",geo_id]))
#     return(feature)

# def geo_id_list_to_featureCollection(list_of_geo_ids):
#     return ee.FeatureCollection(list_of_geo_ids.map(geo_id_to_feature))


### 1. Grab land cover datasets

#### Fetch: GLAD: Global 2000-2020 Land Cover and Land Use Change

For full legend see: https://storage.googleapis.com/earthenginepartners-hansen/GLCLU2000-2020/v2/legend.xlsx

NB all tree cover have values related to height: from 3m to >25m. 
stable tree height values: 25-48 (terra firma); 125-148 (wetland);
height after disturbance values: 49-72 (terra firma); 149-172 (wetland) 
tree gain height values: 73-96 (terra firma); 173-196 (wetland); 

for now including all heights until agreed forest definition 

- value 25 tree stable
- value 49 tree after disturbance 
- value 73 tree gain
- value 248 tree from crop (NB need to investigate: likely plantation but could be secondary)
- value 245 crop
- value 24 vegetation 
- value 200 water 
- value 250 built up


In [7]:
glad_landcover_2020 = ee.Image('projects/glad/GLCLU2020/v2/LCLUC_2020')
landmask = ee.Image("projects/glad/OceanMask").lte(1)
glad_landcover_2020 = glad_landcover_2020.updateMask(landmask);

#visualisation parameters (for original land cover)
visParamMap = {"min":0,"max":255,"palette":["FEFECC","FAFAC3","F7F7BB","F4F4B3","F1F1AB","EDEDA2","EAEA9A","E7E792","E4E48A",
"E0E081","DDDD79","DADA71","D7D769","D3D360","D0D058","CDCD50","CACA48","C6C63F","C3C337","C0C02F","BDBD27","B9B91E","B6B616",
"B3B30E","B0B006","609C60","5C985C","589558","549254","508E50","4C8B4C","488848","448544","408140","3C7E3C","387B38","347834",
"317431","2D712D","296E29","256B25","216721","1D641D","196119","155E15","115A11","0D570D","095409","065106","643700","643a00",
"643d00","644000","644300","644600","644900","654c00","654f00","655200","655500","655800","655a00","655d00","656000","656300",
"666600","666900","666c00","666f00","667200","667500","667800","667b00","ff99ff","FC92FC","F98BF9","F685F6","F37EF3","F077F0",
"ED71ED","EA6AEA","E763E7","E45DE4","E156E1","DE4FDE","DB49DB","D842D8","D53BD5","D235D2","CF2ECF","CC27CC","C921C9","C61AC6",
"C313C3","C00DC0","BD06BD","bb00bb","000003","000004","000005","BFC0C0","B7BDC2","AFBBC4","A8B8C6","A0B6C9","99B3CB","91B1CD",
"89AFD0","82ACD2","7AAAD4","73A7D6","6BA5D9","64A3DB","5CA0DD","549EE0","4D9BE2","4599E4","3E96E6","3694E9","2E92EB","278FED",
"1F8DF0","188AF2","1088F4","0986F7","55A5A5","53A1A2","519E9F","4F9B9C","4D989A","4B9597","499294","478F91","458B8F","43888C",
"418589","3F8286","3D7F84","3B7C81","39797E","37767B","357279","336F76","316C73","2F6970","2D666E","2B636B","296068","285D66",
"bb93b0","B78FAC","B48CA9","B189A6","AE85A2","AA829F","A77F9C","A47B99","A17895","9E7592","9A718F","976E8C","946B88","916885",
"8D6482","8A617F","875E7B","845A78","815775","7D5472","7A506E","774D6B","744A68","714765","de7cbb","DA77B7","D772B3","D46EAF",
"D169AB","CE64A8","CB60A4","C85BA0","C4579C","C15298","BE4D95","BB4991","B8448D","B54089","B23B86","AF3682","AB327E","A82D7A",
"A52976","A22473","9F1F6F","9C1B6B","991667","961264","000000","000000","000000",
"1964EB","1555E4","1147DD","0E39D6","0A2ACF","071CC8","030EC1","0000BA",
"0000BA","040464","0000FF","3051cf","000000","000000","000000","000000",
"000000","000000","000000","000000","000000","000000","000000","000000",
"000000","000000","000000","000000","000000","000000","000000","000000",
"547FC4","4D77BA","466FB1","4067A7","395F9E","335895","335896","335897","ff2828","ffffff","d0ffff","ffe0d0","ff7d00","fac800","c86400",
"fff000","afcd96","afcd96","64dcdc","00ffff","00ffff","00ffff","111133","000000"]};

#trees
glad_landcover_2020_main = glad_landcover_2020.where((glad_landcover_2020.gte(25)).And(glad_landcover_2020.lte(48)), 25)
glad_landcover_2020_main = glad_landcover_2020_main.where((glad_landcover_2020.gte(125)).And(glad_landcover_2020.lte(148)), 25)
# glad_landcover_2020_main = glad_landcover_2020_main.where((glad_landcover_2020.gte(49)).And(glad_landcover_2020.lte(72)), 49)
# glad_landcover_2020_main = glad_landcover_2020_main.where((glad_landcover_2020.gte(149)).And(glad_landcover_2020.lte(172)), 49)
# glad_landcover_2020_main = glad_landcover_2020_main.where((glad_landcover_2020.gte(73)).And(glad_landcover_2020.lte(96)), 73)
# glad_landcover_2020_main = glad_landcover_2020_main.where((glad_landcover_2020.gte(173)).And(glad_landcover_2020.lte(196)), 73)

# #vegetation
# glad_landcover_2020_main = glad_landcover_2020_main.where((glad_landcover_2020.gte(0)).And(glad_landcover_2020.lte(24)), 24)
# glad_landcover_2020_main = glad_landcover_2020_main.where((glad_landcover_2020.gte(100)).And(glad_landcover_2020.lte(124)), 24)
# glad_landcover_2020_main = glad_landcover_2020_main.where((glad_landcover_2020.eq(240)).Or(glad_landcover_2020.eq(249)), 24)

# #cropland
# glad_landcover_2020_main = glad_landcover_2020_main.where((glad_landcover_2020.gte(244)).And(glad_landcover_2020.lte(247)), 244)

# #water
# glad_landcover_2020_main = glad_landcover_2020_main.where((glad_landcover_2020.gte(200)).And(glad_landcover_2020.lte(211)), 200)
# glad_landcover_2020_main = glad_landcover_2020_main.where(glad_landcover_2020.eq(254), 200)

# #built up
# glad_landcover_2020_main = glad_landcover_2020_main.where((glad_landcover_2020.gte(250)).And(glad_landcover_2020.lte(253)), 250)


#### Get stable trees for 2020

In [8]:
glad_stable_tree_2020 = glad_landcover_2020_main.eq(25) #binary stable trees
glad_stable_tree_2020 = set_scale_property_from_image(glad_stable_tree_2020,glad_landcover_2020)

glad_stable_tree_2020_area_km2 = binary_to_area_km2(glad_stable_tree_2020)

glad_stable_tree_2020_area_km2 = set_scale_property_from_image(glad_stable_tree_2020_area_km2,glad_landcover_2020)


#### Fetch: Tropical Moist Forest by JRC
Link: https:#  www.science.org/doi/10.1126/sciadv.abe1603 (paper); https://forobs.jrc.ec.europa.eu/static/tmf/TMF_DataUsersGuide.pdf (dataset description)  

Overview: The transition map shows the spatial distribution of the tropical moist forest at the end of
the year 2022. It depicts the sequential dynamics of changes by providing transition stages
from the first year of the monitoring period to the end of the year 2022 (undisturbed forest,
degradation, deforestation, regrowth, conversion to plantations) and subclasses for each
transition class (period of disturbance, age of regrowth, several types of forest, several
types of degradation and deforestation, change types within the mangroves and tree
plantations).

Reference: C. Vancutsem, F. Achard, J.-F. Pekel, G. Vieilledent, S. Carboni, D. Simonetti, J. Gallego,
L.E.O.C. Aragão, R. Nasi. Long-term (1990-2019) monitoring of forest cover changes in the
humid tropics. Science Advances 2021


Legend: 
- value 10. Undisturbed Tropical Moist Forest (TMF) 
- value 20. Degraded TMF 
- value 30. TMF regrowth 
- value 41. Deforested land - Forest converted to tree plantations 
- value 42. Deforested Land - Forest converted to water
- value 43. Deforested Land - Forest converted to other LC
- value 50. Ongoing deforestation or degradation (2019-2021)
- value 60. Permanent or Seasonal Water  (Pekel et al. 2016 & updates for years 2015-2021) 
- value 70. Other land cover (including afforestation)

In [9]:
JRC_TMF_transitions_raw = ee.ImageCollection('projects/JRC/TMF/v1_2021/TransitionMap_Subtypes')

JRC_TMF_transitions = JRC_TMF_transitions_raw.mosaic() ### NB check why 2021?

JRC_TMF_transitions_main = JRC_TMF_transitions.where((JRC_TMF_transitions.gte(10))and(JRC_TMF_transitions.lte(12)), 10) 
#JRC_TMF_transitions_main = JRC_TMF_transitions_main.where((JRC_TMF_transitions.gte(21))and(JRC_TMF_transitions.lte(26)), 20)
#JRC_TMF_transitions_main = JRC_TMF_transitions_main.where((JRC_TMF_transitions.gte(61))and(JRC_TMF_transitions.lte(62)), 20)
# JRC_TMF_transitions_main = JRC_TMF_transitions_main.where((JRC_TMF_transitions.gte(31))and(JRC_TMF_transitions.lte(33)), 30)
# JRC_TMF_transitions_main = JRC_TMF_transitions_main.where((JRC_TMF_transitions.gte(63))and(JRC_TMF_transitions.lte(64)), 30)
# JRC_TMF_transitions_main = JRC_TMF_transitions_main.where((JRC_TMF_transitions.gte(81))and(JRC_TMF_transitions.lte(86)), 41)
# JRC_TMF_transitions_main = JRC_TMF_transitions_main.where((JRC_TMF_transitions.gte(73))and(JRC_TMF_transitions.lte(74)), 42)
# JRC_TMF_transitions_main = JRC_TMF_transitions_main.where((JRC_TMF_transitions.gte(41))and(JRC_TMF_transitions.lte(42)), 43)
# JRC_TMF_transitions_main = JRC_TMF_transitions_main.where((JRC_TMF_transitions.gte(65))and(JRC_TMF_transitions.lte(66)), 43)
# JRC_TMF_transitions_main = JRC_TMF_transitions_main.where((JRC_TMF_transitions.gte(51))and(JRC_TMF_transitions.lte(54)), 50)
# JRC_TMF_transitions_main = JRC_TMF_transitions_main.where((JRC_TMF_transitions.eq(67)),50)
# JRC_TMF_transitions_main = JRC_TMF_transitions_main.where((JRC_TMF_transitions.gte(71))and(JRC_TMF_transitions.lte(72)), 60)
# JRC_TMF_transitions_main = JRC_TMF_transitions_main.where((JRC_TMF_transitions.gte(91)), 70)

##### Get undisturbed trees for 2020

In [10]:
# JRC_TMF_transitions_trees = JRC_TMF_transitions_main.remap([10,20,30,41,42,43,50,60,70],
JRC_TMF_undisturbed_2020 = JRC_TMF_transitions_main.eq(10) # binary undisturbed trees

#getting scale from first band of raw image and adding to properties of final composite
JRC_TMF_undisturbed_2020 = set_scale_property_from_image(JRC_TMF_undisturbed_2020,
                                                                  JRC_TMF_transitions_raw.first())

#area (NB could be done at end in zonal stats)
JRC_TMF_undisturbed_2020_area_km2 = binary_to_area_km2(JRC_TMF_undisturbed_2020)

JRC_TMF_undisturbed_2020_area_km2 = set_scale_property_from_image(JRC_TMF_undisturbed_2020_area_km2,
                                                                  JRC_TMF_transitions_raw.first())

print (JRC_TMF_undisturbed_2020_area_km2.get("scale").getInfo())

30.000000000000004


In [11]:
##checks
# m = geemap.Map()
# m.addLayer(JRC_TMF_undisturbed_2020.randomVisualizer(),{},'JRC_TMF_undisturbed_2020')
# m.addLayer(JRC_TMF_transitions_main.eq(10).randomVisualizer(),{},'TMF-simple')

# m.addLayer(roi,{},'field')

# m.centerObject(roi)
# m

#### Fetch: Global Forest Change 
Link: https://www.science.org/doi/10.1126/science.1244693 (paper); https://storage.googleapis.com/earthenginepartners-hansen/GFC-2022-v1.10/download.html (user notes v1.10)

Citation: Hansen, M. C., P. V. Potapov, R. Moore, M. Hancher, S. A. Turubanova, A. Tyukavina, D. Thau, S. V. Stehman, S. J. Goetz, T. R. Loveland, A. Kommareddy, A. Egorov, L. Chini, C. O. Justice, and J. R. G. Townshend. "High-Resolution Global Maps of 21st-Century Forest Cover Change." Science 342 (15 November): 850-53. 10.1126/science.1244693.

Overview: Results from time-series analysis of Landsat images in characterizing global forest extent and change from 2000 through 2022.


N.B. Next GFC update should give tree cover in 2020. Currently showing tree cover is for 2000 with loss pixels between 2001 and 2020 removed. 


In [12]:
gfc = ee.Image("UMD/hansen/global_forest_change_2022_v1_10")

gfc_treeCover_2000 = gfc.select(['treecover2000']) #get tree cover in 2000

gfc_loss_2001_2020 = gfc.select(['lossyear']).lte(20) # get loss pixels since 2000 and up to and including 2020

gfc_treeCover_2020 = gfc_treeCover_2000.where(gfc_loss_2001_2020.eq(1),0) # remove loss from original tree cover ot approximate remaining percentage cover


#### Get treecover in 2020

In [13]:
gfc_treeCover_2020_binary=gfc_treeCover_2020.gt(10) #FAO 10% definition...

gfc_treeCover_2020_binary = set_scale_property_from_image(gfc_treeCover_2020_binary,gfc)
## TO DO think about gain and 2010 cover

gfc_treeCover_2020_area_km2 = binary_to_area_km2(gfc_treeCover_2020_binary)

gfc_treeCover_2020_area_km2 = set_scale_property_from_image(gfc_treeCover_2020_area_km2,gfc)

# Map = geemap.Map()
# Map.addLayer(gfc_treeCover_2020_binary,{'min':0,'max':1,'palette':["blue","green"]})
# Map

#### Get area of all valid pixels

In [14]:
total_roi_area_gfc_km2 = binary_to_area_km2(gfc_treeCover_2020_binary.gte(0))
total_roi_area_gfc_km2 = set_scale_property_from_image(total_roi_area_gfc_km2,gfc,0,True)

# Map = geemap.Map()
# Map.addLayer(total_roi_area_gfc_km2)
# Map

template_band_index:  0
scale (m):  27.829872698318393


#### Fetch: ESRI 10m Annual Land Use Land Cover (2017-2022)¶
- Link: https://gee-community-catalog.org/projects/S2TSLULC/?h=esri

- Overview: Time series of annual global maps of land use and land cover (LULC). It currently has data from 2017-2021. The maps are derived from ESA Sentinel-2 imagery at 10m resolution. Each map is a composite of LULC predictions for 9 classes throughout the year in order to generate a representative snapshot of each year. This dataset was generated by Impact Observatory, who used billions of human-labeled pixels (curated by the National Geographic Society) to train a deep learning model for land classification. This map uses an updated model from the 10-class model and combines Grass(formerly class 3) and Scrub (formerly class 6) into a single Rangeland class (class 11). The original Esri 2020 Land Cover collection uses 10 classes (Grass and Scrub separate) and an older version of the underlying deep learning model. 

- Reference:  Karra, Kontgis, et al. “Global land use/land cover with Sentinel-2 and deep learning.” IGARSS 2021-2021 IEEE International Geoscience and Remote Sensing Symposium. IEEE, 2021.
- Legend (remapped values and hex code):
- 1	Water	#1A5BAB
- 2	Trees	#358221
- 3	Flooded Vegetation	#87D19E
- 4	Crops	#FFDB5C
- 5	Built Area	#ED022A
- 6	Bare Ground	#EDE9E4
- 7	Snow/Ice	#F2FAFF
- 8	Clouds	#C8C8C8
- 9	Rangeland	#C6AD8D

In [15]:
def remapper(image):
    remapped = image.remap([1,2,4,5,7,8,9,10,11],[1,2,3,4,5,6,7,8,9])
    return remapped

esri_lulc10 = ee.ImageCollection("projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m_TS");

#esri_lulc10 = ee.ImageCollection("projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m")/old link
esri_lulc10_2020 = esri_lulc10.filterDate('2020-01-01','2020-12-31').map(remapper).mosaic()


##### Get trees


In [16]:
esri_trees_2020 = esri_lulc10_2020.eq(2) #get trees    NB check flooded veg class

esri_trees_2020 = set_scale_property_from_image(esri_trees_2020,esri_lulc10.first(),0)

esri_trees_2020_area_km2 = binary_to_area_km2(esri_trees_2020)

esri_trees_2020_area_km2 = set_scale_property_from_image(esri_trees_2020_area_km2,esri_lulc10.first(),0,verbose=True)

# Map = geemap.Map()
# Map.addLayer(esri_trees_2020,{'min':0,'max':1,'palette':["blue","green"]})
# Map

template_band_index:  0
scale (m):  10


In [17]:
total_roi_area_esri_km2 = binary_to_area_km2(esri_trees_2020.gte(0))
total_roi_area_esri_km2 = set_scale_property_from_image(total_roi_area_esri_km2,esri_lulc10.first(),0,True)

# Map = geemap.Map()
# Map.addLayer(total_roi_area_km2)
# Map

template_band_index:  0
scale (m):  10


### 2. Fetch a Field (public)

In [18]:
CIV_ids = ['0520cfac98fbc1bd7952b1c07a9f6983b83625722b6f665ea83ac9aad3512918',
           'b84f55de2b7f3c77d1cbeb8b026a1b29be42d8b08d92058c9143e0556456820f',
           'b7c15efb6e3c63fcfe649a2d994973a6f5caa844f720f0edb7cf24f6a6c3c1b3',
            'fa2aff0d60cf1bc0e1f1dd4b91daf932940c31c021ca1b84f5b9445855eef02f']

GHA_ids = ['88bec54ad04804f5b1fafbc131266640a129be2840fa6797cda358d7e831b907', 
'ef2f7c46fbe4fc892fdb81f9a31c9c507b9f1e4548504247dcbbab28cf8e436c',
'97408ef7bdac487e4a42e4abf20492b786310889fd4b0478603e2d0004c40bfb']

IDN_ids = ['c288d6c94efa9011c0e3452af9f7fa0941661377030e10d29c68764617f9816d', 
       '1a41a309ae2387f36a604c9a6c81887e64357a7f61d228758e23ef766286fcd7',
       '1a4472dc40700ef33f931863f58d444f243d64418616678fcf85c57e1f4bbf45',
       '8e2accea7ddbb84b7f6001e00bcb60f57f563c80633b53859993522a6f05727a']

all_geo_ids= CIV_ids+GHA_ids+IDN_ids

print (all_geo_ids)

['0520cfac98fbc1bd7952b1c07a9f6983b83625722b6f665ea83ac9aad3512918', 'b84f55de2b7f3c77d1cbeb8b026a1b29be42d8b08d92058c9143e0556456820f', 'b7c15efb6e3c63fcfe649a2d994973a6f5caa844f720f0edb7cf24f6a6c3c1b3', 'fa2aff0d60cf1bc0e1f1dd4b91daf932940c31c021ca1b84f5b9445855eef02f', '88bec54ad04804f5b1fafbc131266640a129be2840fa6797cda358d7e831b907', 'ef2f7c46fbe4fc892fdb81f9a31c9c507b9f1e4548504247dcbbab28cf8e436c', '97408ef7bdac487e4a42e4abf20492b786310889fd4b0478603e2d0004c40bfb', 'c288d6c94efa9011c0e3452af9f7fa0941661377030e10d29c68764617f9816d', '1a41a309ae2387f36a604c9a6c81887e64357a7f61d228758e23ef766286fcd7', '1a4472dc40700ef33f931863f58d444f243d64418616678fcf85c57e1f4bbf45', '8e2accea7ddbb84b7f6001e00bcb60f57f563c80633b53859993522a6f05727a']


In [19]:
# geo_id = all_geo_ids[0] ##select geo_id from above using index
# print (geo_id) #"ef2f7c46fbe4fc892fdb81f9a31c9c507b9f1e4548504247dcbbab28cf8e436c" #"c288d6c94efa9011c0e3452af9f7fa0941661377030e10d29c68764617f9816d" # geo id for the field to fetch

In [20]:
# S2 indexes of 8 and 15 are used here for the geo data
# res = session.get(asset_registry_base + f"/fetch-field/{geo_id}?s2_index=8,15") # s2 index are indexes for which we need S2 cell tokens
# poly_wkt = res.json()['Geo Data']['wkt']
# print(poly_wkt)

#### Transform geometry into a GEE object

In [21]:
###  TO FIX: API issues when creating a feature collection

# f1 = geo_id_to_feature(all_geo_ids[1])##use function to create feature
# f2 = geo_id_to_feature(all_geo_ids[2])
# output_list = (all_geo_ids).map(get_json_from_geo_id)
# print(all_geo_ids[2])
# print(f1.getInfo()) #check feature
# print(f2.getInfo()) ##gets errors when multiple features

#### Making into feature collection 

In [35]:
def geo_id_to_feature(geo_id):
    res = session.get(asset_registry_base + f"/fetch-field/{geo_id}?s2_index=") # s2 index are indexes for which we need S2 cell token
    poly_json = res.json()['Geo JSON']['geometry']['coordinates']
    feature = json_to_feature_with_id(poly_json,geo_id)
    return feature

def json_to_feature_with_id(poly_json,geo_id):
    return ee.Feature(ee.Geometry.Polygon(poly_json),ee.Dictionary(["geo_id",geo_id]))

def geo_id_to_json(geo_id):
    res = session.get(asset_registry_base + f"/fetch-field/{geo_id}?s2_index=") # s2 index are indexes for which we need S2 cell token
    poly_json = res.json()['Geo JSON']['geometry']['coordinates']
    return poly_json

def geo_id_list_to_feature_collection(list_of_geo_ids):
    out_fc_list = []
    if isinstance(list_of_geo_ids, list):
        for geo_id in list_of_geo_ids:
            feature = geo_id_to_feature(geo_id)
            out_fc_list.append(feature)
    else:
        geo_id = list_of_geo_ids
        feature = geo_id_to_feature(geo_id)
        out_fc_list.append(feature)
    return ee.FeatureCollection(out_fc_list)

roi = geo_id_list_to_feature_collection(all_geo_ids)

#checks 
print ("Count of geo ids in list: ", len(all_geo_ids))
print ("Count of features in FeatureCollection: ", roi.size().getInfo())

Count of geo ids in list:  11
Count of features in FeatureCollection:  1


#### Add area property to feature(s)

Temporary code: This won't match pixel based areas, but useful for checks for now.

In [36]:
roi = roi.map(add_area_km2_property_to_feature)

##checks
##print ("first feature area",roi.first().get("area_km2").getInfo())

#### Compute statistics

Calculating zonal statistics for continuous data (e.g tree cover) within polygon(s)

Save as CSV in long format

In [37]:
continuous_image_dict = {"total_roi_area_esri_km2":total_roi_area_esri_km2,
                         # "total_roi_area_gfc_km2":total_roi_area_gfc_km2,
                         "GFC_2020":gfc_treeCover_2020_area_km2,
                         "TMF_undisturbed_2020":JRC_TMF_undisturbed_2020_area_km2,
                        "esri_trees_2020":esri_trees_2020_area_km2,
                         "glad_stable_tree_2020":glad_stable_tree_2020_area_km2}
                         
out_path = os.path.join('/home/sepal-user/fdap/')

out_csv_name = 'continuous_stats.csv' 

out_file=out_path+out_csv_name

header = True

##get zonal stats csvs, add datasets name, and append all into one csv  
for name, image in continuous_image_dict.items():

    stats_csv = os.path.join(out_path+name+'.csv')
    
    out_scale = image.get("scale").getInfo()

    geemap.zonal_stats(in_value_raster=image, in_zone_vector=roi, out_file_path=stats_csv, 
                        statistics_type='SUM',scale=out_scale)

    df = pd.read_csv(stats_csv)
    
    df["dataset_name"] =name
       
    df.to_csv(path_or_buf=stats_csv,index=False) #df['Dataset']=name
    
    mode = 'w' if header else 'a'
    
    df.to_csv(path_or_buf=out_file,mode = mode,index=False,header=header) #df['Dataset']=name
    
    os.remove(stats_csv)
    
    header = False


Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /home/sepal-user/fdap/total_roi_area_esri_km2.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /home/sepal-user/fdap/GFC_2020.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /home/sepal-user/fdap/TMF_undisturbed_2020.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /home/sepal-user/fdap/esri_trees_2020.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /home/sepal-user/fdap/glad_stable_tree_2020.csv


##### Reformat long to wide

In [40]:
df_long_format = pd.read_csv(out_file)
df_long_format
# df_long_format.pivot(index=, columns='geoid', values='sum')
# df.set_index(['geoid', 'system:index', 'area_km2'], append=False)

df2 = df_long_format.pivot_table(index=['geo_id','system:index'], columns='dataset_name', values='sum')#.reset_index()
df2


,dataset_name,GFC_2020,TMF_undisturbed_2020,esri_trees_2020,glad_stable_tree_2020,total_roi_area_esri_km2
geo_id,system:index,,,,,
b84f55de2b7f3c77d1cbeb8b026a1b29be42d8b08d92058c9143e0556456820f,0,0.015586,0.0,0.019807,0.011034,0.019807


Calculating zonal statistics for categorical data (e.g. land cover) within polygon 
 - to tidy NB more efficient for proportions if only using binary data..

In [39]:
group_stats = os.path.join('/home/sepal-user/fdap/poly_group_stats.csv') 

#input_image = JRC_TMF_transitions_main 
input_image = glad_stable_tree_2020#_main

# out_scale= get_scale_from_image(input_image)
out_scale = input_image.get("scale").getInfo()

print(out_scale)

geemap.zonal_stats_by_group(in_value_raster=input_image, in_zone_vector=roi, out_file_path=group_stats, statistics_type='SUM', 
                     decimal_places=0, denominator=1000.0, scale=out_scale, crs=None, crs_transform=None, 
                     best_effort=False, max_pixels=10000000.0, tile_scale=1.0, return_fc=False, 
                     verbose=True, timeout=300, proxies=None)

group_stats_table = pd.read_csv(group_stats)
group_stats_table

#TO DO - think about equal area vs unprojected


27.829872698318393
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to /home/sepal-user/fdap/poly_group_stats.csv


,Class_0,Class_1,Class_sum,area_km2,system:index,geo_id
0,9,11,20.0,0.019902,0,b84f55de2b7f3c77d1cbeb8b026a1b29be42d8b08d9205...


#### Display layers

NB add legends


In [ ]:
m = geemap.Map()

In [ ]:
vis_params = {min:0,max:1,'palette':["white","green"]}

m.addLayer(gfc_treeCover_2020_binary,vis_params,'gfc_treeCover_2020_binary')
m.addLayer(esri_trees_2020,vis_params,"esri_trees_2020")
m.addLayer(glad_stable_tree_2020,vis_params,'glad_stable_tree_2020')
m.addLayer(JRC_TMF_undisturbed_2020,vis_params,'JRC_TMF_undisturbed_2020')
# m.addLayer(JRC_TMF_transitions.eq(10).randomVisualizer(),{},'TMF-simple')

m.addLayer(roi,{},'field')

m.centerObject(roi)


In [ ]:
m

#### Alternative stats
Using reduceRegion...

In [ ]:
# Define the class names
class_names = ['TMF_undis', 'TMF_degrad', 'TMF_regrow', 'Defor_plantation', 'Defor_water', 'Defor_land','Defor_active','water','other_land']

# Create a function to count pixels for each class
def count_pixels(image, roi):
    counts = image.reduceRegion(
        reducer=ee.Reducer.frequencyHistogram(),
        geometry=roi,
        scale=90,
        maxPixels=1e9
    )
    
    # Convert pixel counts to a feature with properties
    feature = ee.Feature(None, counts)
    
    return feature

# Apply the count_pixels function to the classified image
pixel_counts_feature = count_pixels(tmf, roi)

# Create a feature collection with a single feature
fc_pix = ee.FeatureCollection([pixel_counts_feature])


fc_pix_df = geemap.ee_to_pandas(fc_pix)##needs more work
fc_pix_df




### Logout (protected)

In [ ]:
res = session.get(asset_registry_base + "/logout")
print(res.json())
res = session.get(user_registry_base + "/logout", cookies=session.cookies)
session.headers.clear()

### Checking if Logged out correctly

In [ ]:
# Confirming the logout from Asset Registry by requesting a Protected route
req_body = {
    "latitude": 31.47704430446457,
    "longitude": 74.37510786779589
}
res = session.post(asset_registry_base + "/fetch-fields-for-a-point", json=req_body)
print(res.json())

### Get all Domains (public)

In [ ]:
# Fetching all the domains from the User Registry
res = session.get(asset_registry_base + "/domains")
print(res.json())